In [5]:
import numpy as np
import pandas as pd
from os import makedirs
from os.path import join, exists
#from nilearn.input_data import NiftiLabelsMasker
from nilearn.connectome import ConnectivityMeasure
from nilearn.plotting import plot_anat, plot_roi
import bct
#from nipype.interfaces.fsl import InvWarp, ApplyWarp
import datetime

subjects = ['101', '102', '103', '104', '106', '107', '108', '110', '212', '213',
            '214', '215', '216', '217', '218', '219', '320', '321', '322', '323',
            '324', '325', '327', '328', '329', '330', '331', '332', '333', '334',
            '335', '336', '337', '338', '339', '340', '341', '342', '343', '344',
            '345', '346', '347', '348', '349', '350', '451', '452', '453', '455',
            '456', '457', '458', '459', '460', '462', '463', '464', '465', '467',
            '468', '469', '470', '502', '503', '571', '572', '573', '574', '575',
            '577', '578', '579', '580', '581', '582', '584', '585', '586', '587',
            '588', '589', '590', '591', '592', '593', '594', '595', '596', '597',
            '598', '604', '605', '606', '607', '608', '609', '610', '611', '612',
            '613', '614', '615', '616', '617', '618', '619', '620', '621', '622',
            '623', '624', '625', '626', '627', '628', '629', '630', '631', '633',
            '634']
#subjects = ['101', '102']


sink_dir = '/Users/katherine/Dropbox/Projects/physics-retrieval/data/output'

shen = '/home/kbott006/physics-retrieval/shen2015_2mm_268_parcellation.nii.gz'
craddock = '/home/kbott006/physics-retrieval/craddock2012_tcorr05_2level_270_2mm.nii.gz'
masks = {'shen2015': shen, 'craddock2012': craddock}

sessions = [0,1]
sesh = ['pre', 'post']
tasks = ['rest']

kappa_upper = 0.21
kappa_lower = 0.31

lab_notebook_dir = sink_dir
index = pd.MultiIndex.from_product([subjects, sessions], names=['subject', 'session'])
lab_notebook = pd.DataFrame(index=index, columns=['start', 'end', 'errors'])

correlation_measure = ConnectivityMeasure(kind='correlation')


index = pd.MultiIndex.from_product([subjects, sessions, tasks, masks.keys()], names=['subject', 'session', 'task', 'mask'])
df = pd.DataFrame(columns=['lEff1', 'clustCoeff1'], index=index, dtype=np.float64)



In [6]:
for subject in subjects:
    for session in sessions:
        lab_notebook.at[(subject, session),'start'] = str(datetime.datetime.now())
        for task in tasks:
            for mask in masks.keys():
                try:
                    #shen_masker = NiftiLabelsMasker(xfmd_masks['shen2015'], background_label=0, standardize=True, detrend=True,t_r=3.)
                    #craddock_masker = NiftiLabelsMasker(xfmd_masks['craddock2012'], background_label=0, standardize=True, detrend=True,t_r=3.)

                    #confounds = '/home/data/nbc/physics-learning/anxiety-physics/output/{1}/{0}/{0}_confounds.txt'.format(subject, sesh[session])
                    #epi_data = join(data_dir, subject, 'session-{0}'.format(session), 'resting-state/resting-state-0/endor1.feat', 'filtered_func_data.nii.gz')

                    #shen_ts = shen_masker.fit_transform(epi_data, confounds)
                    #shen_corrmat = correlation_measure.fit_transform([shen_ts])[0]
                    #np.savetxt(join(sink_dir, sesh[session], subject, '{0}-session-{1}-rest_network_corrmat_shen2015.csv'.format(subject, session)), shen_corrmat, delimiter=",")
                    corrmat = np.genfromtxt(join(sink_dir, '{0}-session-{1}-{2}_network_corrmat_{3}.csv'.format(subject, session, task, mask)), delimiter=",")
                    print(corrmat.shape)
                    #craddock_ts = craddock_masker.fit_transform(epi_data, confounds)
                    #craddock_corrmat = correlation_measure.fit_transform([craddock_ts])[0]
                    #np.savetxt(join(sink_dir, sesh[session], subject, '{0}-session-{1}-rest_network_corrmat_craddock2012.csv'.format(subject, session)), craddock_corrmat, delimiter=",")

                    ge_s = []
                    ge_c = []
                    
                    md_s = []
                    md_c = []
                    for p in np.arange(kappa_upper, kappa_lower, 0.02):
                        thresh = bct.threshold_proportional(corrmat, p, copy=True)

                        #network measures of interest here
                        #global efficiency
                        ge = bct.efficiency_wei(thresh, local=True)
                        ge_s.append(ge)

                        #modularity
                        md = bct.clustering_coef_wu(thresh)
                        md_s.append(md)

                    ge_s = np.asarray(ge_s)
                    md_s = np.asarray(md_s)
                    leff = np.trapz(ge_s, dx=0.01, axis=0)
                    print('local efficiency:', leff[0])
                    ccoef = np.trapz(md_s, dx=0.01, axis=0)
                    for j in np.arange(1, 270):
                        df.at[(subject, session, task, mask), 'lEff{0}'.format(j)] = leff[j-1]
                        df.at[(subject, session, task, mask), 'clustCoeff{0}'.format(j)] = ccoef[j-1]
                    #df.to_csv(join(sink_dir, 'resting-state_graphtheory_shen+craddock.csv'), sep=',')
                    lab_notebook.at[(subject, session),'end'] = str(datetime.datetime.now())
                except Exception as e:
                    print(e, subject, session)
                    lab_notebook.at[(subject,session),'errors'] = [e, str(datetime.datetime.now())]
        df.to_csv(join(sink_dir, 'resting-state_nodal-graphtheory_shen+craddock.csv'), sep=',')

df.to_csv(join(sink_dir, 'resting-state_nodal-graphtheory_shen+craddock_{0}.csv'.format(str(datetime.datetime.today()))), sep=',')
lab_notebook.to_csv(join(lab_notebook_dir, 'LOG_resting-state-graph-theory_{0}.csv'.format(str(datetime.datetime.now()))))


(268, 268)


KeyboardInterrupt: 

In [7]:
df

lEff1  clustCoeff1     lEff2  \
subject session task mask                                            
101     0       rest shen2015      0.015549     0.010281  0.015645   
                     craddock2012  0.014796     0.009625  0.015527   
        1       rest shen2015      0.013322     0.008827  0.013024   
                     craddock2012  0.012358     0.007088  0.013572   
102     0       rest shen2015      0.012596     0.006547  0.015166   
                     craddock2012  0.014116     0.008609  0.014109   
        1       rest shen2015      0.013338     0.010528  0.014416   
                     craddock2012  0.013033     0.007901  0.012682   

                                   clustCoeff2     lEff3  clustCoeff3  \
subject session task mask                                               
101     0       rest shen2015         0.010683  0.015480     0.010683   
                     craddock2012     0.010485  0.014902     0.009587   
        1       rest shen2015         0.007457  0.010433     0.005435   
                     craddock2012     0.007553  0.014256     0.009402   
102     0       rest shen2015         0.011405  0.011679     0.006382   
                     craddock2012     0.008001  0.013043     0.006828   
        1       rest shen2015         0.010408  0.012551     0.008764   
                     craddock2012     0.006692  0.012305     0.006947   

                                      lEff4  clustCoeff4     lEff5  \
subject session task mask                                            
101     0       rest shen2015      0.014675     0.009112  0.015137   
                     craddock2012  0.015684     0.009948  0.014911   
        1       rest shen2015      0.012950     0.006694  0.013472   
                     craddock2012  0.014979     0.009820  0.014211   
102     0       rest shen2015      0.014788     0.008425  0.014091   
                     craddock2012  0.014274     0.008472  0.013974   
        1       rest shen2015      0.014417     0.009188  0.012659   
                     craddock2012  0.012337     0.006171  0.011989   

                                   clustCoeff5  ...   lEff263  clustCoeff263  \
subject session task mask                       ...                            
101     0       rest shen2015         0.010257  ...  0.014103       0.007429   
                     craddock2012     0.010966  ...  0.015884       0.012118   
        1       rest shen2015         0.009003  ...  0.013135       0.007038   
                     craddock2012     0.009712  ...  0.011283       0.006415   
102     0       rest shen2015         0.008163  ...  0.014159       0.007720   
                     craddock2012     0.007585  ...  0.013293       0.008531   
        1       rest shen2015         0.006967  ...  0.011720       0.007192   
                     craddock2012     0.007169  ...  0.012386       0.007506   

                                    lEff264  clustCoeff264   lEff265  \
subject session task mask                                              
101     0       rest shen2015      0.014857       0.008619  0.015419   
                     craddock2012  0.015130       0.010031  0.015037   
        1       rest shen2015      0.014524       0.009387  0.014150   
                     craddock2012  0.014970       0.009650  0.014700   
102     0       rest shen2015      0.012975       0.007398  0.013651   
                     craddock2012  0.014713       0.010450  0.015120   
        1       rest shen2015      0.010762       0.006979  0.013512   
                     craddock2012  0.012920       0.007257  0.013474   

                                   clustCoeff265   lEff266  clustCoeff266  \
subject session task mask                                                   
101     0       rest shen2015           0.010161  0.014943       0.009109   
                     craddock2012       0.009521  0.014491       0.009006   
        1       rest shen2015           0.010244  0.014047       0.009465  

In [10]:
for j in np.arange(1, 269):
    print(ccoef[j-1])

0.007454690022501217
0.007900775309480372
0.0066918149977028265
0.0069473206797935234
0.0061713957389498125
0.007169088245703788
0.007405656577953213
0.007198982729770417
0.006381464545964363
0.008434875432099458
0.007322785219902214
0.00597890874127783
0.010217386515798819
0.01151190372943397
0.006403568750196864
0.007708082284526093
0.010005592207775492
0.005331474734393296
0.010359476804879897
0.007097862545142596
0.00544149690951881
0.007480607090201493
0.009793320202272731
0.008607460556293738
0.006706738646748877
0.007154021121443947
0.009111038566471434
0.009318180823282118
0.007436189354768462
0.007428551266208059
0.006844995047909842
0.00619398333767998
0.008455564433308433
0.006803008123760667
0.0074765325509648355
0.009170952036467514
0.007564604806441582
0.009067524648310916
0.007418657765489023
0.009978812081225908
0.006468073406582229
0.010578806984904522
0.009024685493513965
0.011516528515934982
0.009027238817043464
0.009759536357200373
0.006844007866736957
0.00562761011